In [ ]:
# BFS Template
from collections import deque
from typing import Optional
class TreeNode:
    def __init__(self, val = 0):
        self.val = val
        self.left = None
        self.right = None

class Solution:
    '''
    @param root: A Tree
    @return: Level order a list of lists of integer
    '''
    def levelOrder(self, root: Optional[TreeNode]) -> list[list[int]]:
        if root is None:
            return []

        queue = deque([root])
        result = []
        
        while queue:
            level = []
            for _ in range(len(queue)):
                node = queue.popleft()
                level.append(node.val)
                if node.left:
                    queue.append(node.left) 
                if node.right:
                    queue.append(node.right)
            
            result.append(level)
        
        return result



In [ ]:
# Serialize and Deserialize Template
class Solution:
    def serialize(self, root):
        if root is None:
            return "{}"

        queue = [root]
        index = 0
        while index < len(queue):
            if queue[index] is not None:
                queue.append(queue[index].left)
                queue.append(queue[index].right)
            index += 1
        
        # 不刪也不會錯
        while queue[-1] is None:
            queue.pop()
        
        return '{%s}' % ','.join([str(node.val) if node is not None else '#'
                                 for node in queue])
    
    def deserialize(self, data):
        data = data.strip('\n')
        
        if data == '{}':
            return None
        
        # Start value
        vals = data[1:-1].split(',')
        
        root = TreeNode(int(vals[0]))
        queue = [root]
        isLeftChild = True
        index = 0
        
        # Since the first val has been assigned as root, val start from vals[1]
        for val in vals[1:]:
            if val is not '#':
                node = TreeNode(int(val))
                if isLeftChild:
                    # 這邊仍然在處理 queue[index]: 根節點root，還需要處理right，所以index還不能增加
                    queue[index].left = node
                else:
                    queue[index].right = node
                queue.append(node)
            
            if not isLeftChild:
                index += 1
            isLeftChild = not isLeftChild
        
        return root
            

In [6]:
# Clone Graph
# Definition for a Node.
class Node:
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

from typing import Optional
from collections import deque

class Solution:
    """
    @param: node: A undirected graph node
    @return: A undirected graph path
    """
    def cloneGraph(self, node: Optional['Node']) -> Optional['Node']:
        # root = node
        
        if node is None:
            return node
        
        # Use BFS algorithm to traverse the graph and get all nodes
        nodes = self.getNodes(node)
        
        # Copy node, store the old -> new mapping information in a has map
        # 建立映射（只建節點，不連邊）
        mapping = {}
        for cur in nodes:
            # 原始值和clone之間的連結，先做每一個的val，還沒做鄰居
            mapping[cur] = Node(cur.val)
        
        # Copy neighbors(edges)
        for cur in nodes:
            new_node = mapping[cur]   # 最多n次
            #最壞n次，把n-1個點都連完
            for neighbor in cur.neighbors:
                # 使用 原始值和clone之間的連結 把值抓來
                new_neighbor = mapping[neighbor]        # 不是n*m，是這兩個主體執行多少次決定的，這兩個最多執行2m次O(m) 
                new_node.neighbors.append(new_neighbor) # m不是每個點有多少鄰邊，m是所有點有多少鄰邊
        
        return mapping[node] 
        
    ############# 兩個的time complexity 都是O(N + M), N為點數, M為邊數
    
    def getNodes(self, node: Optional[Node]):
        # 把所有起點放到queue裡
        # 並標記為訪問過了
                      # 只有起點
        queue = deque([node])
        visited = set([node])
        
        # 2. while queue 不空
        while queue:
            # pop出queue的第一個點
            # 把所有的鄰居都扔到隊列裡標記為訪問過
            head = queue.popleft()
            for neighbor in head.neighbors:
                # A 比 B 的寫法好，因為如果B還有多行code，會有縮進的問題，主體部分的縮進太長
                if neighbor in visited:
                    continue
                # 這個visited一定要在同層裡面就操作，不然會重複，造成超時
                visited.add(neighbor)
                queue.append(neighbor)
                # B
                # if neighbor not in visited:
                #     visited.add(neighbor)
                #     ...
                #     ...
                #     ...
                #     queue.append(neighbor)
                    
        return visited
        
# if __name__ == '__main__':
#     Node[1].neighbors = [Node[2], Node[4]]
#     Node[2].neighbors = [Node[1], Node[3]]
#     Node[3].neighbors = [Node[2], Node[4]]
#     Node[4].neighbors = [Node[1], Node[3]]
#     sol = Solution()
#     result = sol.cloneGraph(Node[1])
#     print(result)

In [9]:
# Word Ladder: Implicit Graph 隱式圖最短路徑
# Level Order Traversal, Shortest Path in Simple Graph
# 隱式圖: 不告訴你什麼是點什麼是邊
# 以這題來說，每個單詞是一個點，如果兩個單詞可以變化，中間連一條邊
 
class Solution:
    def ladderLength(self, start, end, dict):
        # 方法一：建議用方法二
        # queue = deque([start])
        # visited = set([start])
        
        # distance = 0
        # while queue:
        #     distance += 1
        #     for i in range(len(queue)):
        #         word = queue.popleft()
        #         if word == end:
        #             return distance
                
        #         for next_word in self.get_next_words(word):
        #             if next_word not in dict or next_word in visited:
        #                 continue
        #             queue.append(next_word)
        #             visited.add(next_word)
        
        # return 0
    
        # 方法二： 多耗一個空間，但少一個回圈
        dictSet = set(dict)     # LC設定不同，要改成set才不會超時
        if end not in dictSet:  # LC條件
            return 0
        
        queue = deque([start])
        distance = {start: 1}    # 多耗一個空間，但多儲存一個資訊，縮進變少，代碼變少，可讀性變高
        
        while queue:
            word = queue.popleft()
            if word == end:
                return distance[word]
                
            for next_word in self.get_next_words(word):
                if next_word not in dictSet or next_word in distance:
                    continue
                queue.append(next_word)
                distance[next_word] = distance[word] + 1
        
        return 0
    
        """
        N is the total number of words => 10k
        L is the max length of word => 10
        N >> L
        方法一：
        for word in dict:    O(N)
            compare word and curt_word edit distance == 1    O(L)
                find word
        
        O(N * L)   100k
        方法二:
        for position in word: O(L)
            for character to change: 25
            generate new word:     O(L)
            chack word in dict or not     O(L)
        
        O(25 * 2 * L^2) = 5k
        """
    def get_next_words(self, word):
        words = set()     # 避免LC超時
        for i in range(len(word)):
            left, right = word[:i], word[i + 1:]
            for char in 'abcdefghijklmnopqrstuvwxyz':
                # 因為要替換字母，所以當word[i] == char時，要跳過這圈，繼續替換下一個a-z的字母，組合起來看看有沒有在dict裡面
                if word[i] == char:
                    continue
                words.add(left + char + right)
        
        return words

if __name__ == '__main__':
    sol = Solution()
    result = sol.ladderLength("hit", "cog", ["hot","dot","dog","lot","log","cog"])
    print(result)
    result = sol.ladderLength("hit", "cog", ["hot","dot","dog","lot","log"])
    print(result)

5
0


In [ ]:
# Number of Islands: 由點及面Connected Component
# BFS: 較好，time complexicity = O(N)
from collections import deque

class Solution:
    """
    @param grid: a boolean 2D matrix
    @return: an integer
    """
    def numIslands(self, grid) -> int:
        if not grid or not grid[0]:
            return 0

        islands = 0
        visited = set()
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] and (i, j) not in visited:
                    self.bfs(grid, i, j, visited)
                    islands += 1
        
        return islands
    
    def bfs(self, grid, x, y, visited):
        queue = deque([(x, y)])
        visited.add((x, y))
        
        while queue:
            x, y = queue.popleft()   # 下左上右
            for delta_x, delta_y in [(1, 0), (0, -1), (-1, 0), (0, 1)]:
                next_x = x + delta_x
                next_y = y + delta_y
                if not self.is_valid(grid, next_x, next_y, visited):
                    continue
                queue.append((next_x, next_y))
                visited.add((next_x, next_y))
    
    def isvalid(self, grid, x, y, visited) -> bool:
        n, m = len(grid), len(grid[0])
        if not (0 <= x < n and 0 <= y < m):
            return False

        if (x, y) in visited:
            return False
        
        return grid[x][y] == '1'

In [ ]:
# Union Find (強化班影片) 自己看

In [ ]:
# Knight Shortest Path
# 要Speed up -> 隨課教程的雙向寬邊搜索 ->起點往終點做寬搜 同時終點往起點做寬搜
# 給起點給終點的，就可以用雙向寬邊搜索來做


In [ ]:
# Topological Sorting 拓撲排序: 一定用BFS不要用DFS
# ***能夠用BFS解決的問題，一定不要用DFS來做
# ***因為用Recursion實現的DFS可能造成StackOverflow
# (Iteration的DFS一來你不會寫，二來也不好懂)

"""
Definition for a Directed graph node
class DirectedGraphNode:
    def __init__(self, x):
        self.label = x
        self.neighbors = []
"""
class Solution:
    """
    @param graph: A list of Directed graph node
    @return: Any topological order for the given graph
    """
    def topSort(self, graph):
        # key to value -> 說明是一個hash table
        node_to_indegree = self.getIndegree(graph)
        
        # bfs
        order = []
        start_nodes = [n for n in graph if node_to_indegree[n] == 0]
        queue = deque(start_nodes)
        while queue:
            node = queue.popleft()
            order.append(node)
            for neighbor in node.neighbors:
                node_to_indegree[neighbor] -= 1
                # 只可能出現一次0，所以不需要一個hash set來記錄visited
                if node_to_indegree[neighbor] == 0:
                    queue.append(neighbor)
                    
        return order   # 如果要判斷有沒有環，只需要判對最後order的長度有沒有=graph，有的話就是有環
    
    def getIndegree(self, graph):
        node_to_indegree = {x: 0 for x in graph}
        
        for node in graph:
            for neighbor in node.neighbors:
                node_to_indegree[neighbor] += 1
        
        return node_to_indegree
        

In [ ]:
# Alian Dictionary
class Solution:
    def topological_sort(self, graph):
        # before looking into this part of code